### _* Memo_0915_0_

In [1]:
# Configuration
from config import next_round, version_main

version_sub=0
version=f"{version_main}_{version_sub}"
title = "Metrics"
row_length = 150
new_to_old = True
show_mark_down = True
markdown_table_limit_length = 30
# 1179 3,16,18,24,40,44+21. 

In [2]:
## this code was run.
#!pip install keras-tuner

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import math
from datetime import datetime
from tqdm import tqdm
print(f'Current Step #01 [now = {datetime.now()}')
import matplotlib.pyplot as plt
from IPython.display import Markdown


Current Step #01 [now = 2025-09-26 16:41:00.048834


In [3]:
def get_candidated(version: str):
    """ get_candidated """
    query = f'select n0, n1, n2, n3, n4, n5 from candidated where version=\\"{version}\\";'
    metrics = !echo "{query}" | sqlite3 ../db/metrics.db
    if len(metrics) == 1:
        metrics = metrics[0].split('|')
        if len(metrics) == 6:
            return {0: int(metrics[0]),
                    1: int(metrics[1]),
                    2: int(metrics[2]),
                    3: int(metrics[3]),
                    4: int(metrics[4]),
                    5: int(metrics[5])}
    return {}


def set_candidated_level(version: str, level: int):
    """ set_candidated_level """
    """ candidated의 level을 설정합니다. """ 
    up_query = f'update candidated set level={level} where version=\\"{version}\\";'
    !echo "{up_query}" | sqlite3 ../db/metrics.db


def get_candidated_level(version: str):
    """ get_candidated_level """
    """ level을 가져 옵니다. """
    sel_query = f'select level from candidated where version=\\"{version}\\";'
    levels = !echo "{sel_query}" | sqlite3 ../db/metrics.db

    if len(levels) > 0:
        return int(levels[0])
    return 0


def update_candidated(candidated: dict, version: str):
    """ update_candidated """
    sel_query = f'select * from candidated where version=\\"{version}\\";'
    metrics = !echo "{sel_query}" | sqlite3 ../db/metrics.db
    if len(metrics) == 1:
        """ update """
        update_query = 'update candidated set '
        set_kv = []
        for key in candidated.keys():
            set_kv.append(f'n{key}={candidated[key]}')
        update_query += ", ".join(set_kv)
        update_query += f' where version=\\"{version}\\";'
        !echo "{update_query}" | sqlite3 ../db/metrics.db
    else:
        """ insert """
        insert_query = 'insert into candidated ('
        new_key = []
        new_val = []
        for key in candidated.keys():
            new_key.append(f'n{key}')
            new_val.append(f'{candidated[key]}')
        new_key.append('version')
        new_val.append(f'\\"{version}\\"')
        insert_query += f'{", ".join(new_key)} ) values ('
        insert_query += f'{", ".join(new_val)} );'
        !echo "{insert_query}" | sqlite3 ../db/metrics.db


def create_candidated():
    """ create_candidated """
    new_query = 'create table candidated ( ' \
                ' id INTEGER PRIMARY KEY AUTOINCREMENT, ' \
                ' version char[14], ' \
                ' n0 int, ' \
                ' n1 int, ' \
                ' n2 int, ' \
                ' n3 int, ' \
                ' n4 int, ' \
                ' n5 int );'
    !echo "{new_query}" | sqlite3 ../db/metrics.db


def draw_nums(ext_datas, is_pre_data=False, fig_size=(15, 8)):
    """ draw_nums """
    show_cols = [0, 1, 2, 3, 4, 5]
    show_label = False
    show_cnt = 100
    start_pos = row_length - show_cnt
    show_data = []
    for i in range(6):
        end_pos = -1 if is_pre_data else len(row_list_none_last[i])
        show_data.append(row_list_none_last[i][start_pos:end_pos])
    print(ext_datas)
    draw_scatter(show_data, f'line num : {key}', True, True, show_cols, ext_datas, show_label, helper_line_nums=[10, 20, 30, 40], fig_size=fig_size)


# 기준이 되는 수에 대한 DB의 기존 데이터를 출력합니다.
def get_likey(like_query, next_round):
    """ get_likey """
    rows = !echo "{like_query}" | sqlite3 ../db/metrics.db
    cur_next_round = next_round
    if len(rows) > 0:
        for row in rows:
            cols = row.split("|")
            nums = [int(i) for i in cols[2].split(',')]
            nums_str = [f"{int(i): >2}" for i in nums]
            round_diff = 0
            cur_round = int(cols[1])        
            if next_round > 0:
                round_diff = cur_next_round - cur_round
            cur_next_round = cur_round
            print(f"{cur_round: >4} | {','.join(nums_str)} | {sum(nums): >3} | {round_diff: >3}")
    else:
        print('data is empty.')


cur_level = get_candidated_level(version=version)

In [4]:
## 레벨을 설정합니다.
## 변경시에는 아래의 cur_level을 해제하고 실행하여야 합니다.
db_level = get_candidated_level(version=version)
print(f'cur_level = {cur_level}, db_level = {db_level}')
if cur_level != db_level:
    set_candidated_level(version=version, level=cur_level)
    db_level = get_candidated_level(version=version)
    print(f'cur_level = {cur_level}, db_level = {db_level}')
#cur_level = 1

cur_level = 0, db_level = 0


In [5]:
# Display the variable's value in Markdown
contents = f"###### Updated {datetime.now()}\n\n### {title}\n"
query = 'select round, metric, bonus from results'
query += f' order by round {"desc" if new_to_old else "asc"}'
limit_str = f'limit {row_length}'
query += f' {limit_str if row_length > 0 else ""}'
rows = !echo "{query}"| sqlite3 ../db/metrics.db
table_rows = ["|round|numbers|bonus|sum|0|10|20|30|40|",
              "|--|--|--|--|--|--|--|--|--|"
             ]
rows_dict = {1:[], 10:[], 20:[], 30:[], 40:[]}
nums_t = []
sums = []
cur_table_limit = 0
print(f'row\'s len = {len(rows)}')
for row in rows:
    cols = row.split('|')
    if len(cols) == 3:
        num_00 = 0
        num_10 = 0
        num_20 = 0
        num_30 = 0
        num_40 = 0
        nums_list = [int(i) for i in cols[1].split(',')]
        for num in nums_list:
            if num < 10:
                num_00 += 1
            if num >= 10 and num < 20:
                num_10 += 1
            if num >= 20 and num < 30:
                num_20 += 1
            if num >= 30 and num < 40:
                num_30 += 1
            if num >= 40 :
                num_40 += 1
        rows_dict[1].append(num_00)
        rows_dict[10].append(num_10)
        rows_dict[20].append(num_20)
        rows_dict[30].append(num_30)
        rows_dict[40].append(num_40)
        if cur_table_limit < markdown_table_limit_length:
            nums=[f"{i:02}" for i in nums_list] 
            table_rows.append(f"|{cols[0]}|{",".join(nums)}|{cols[2]}|{sum(nums_list)}|{num_00}|{num_10}|{num_20}|{num_30}|{num_40}|")
            cur_table_limit += 1
        nums_t.append(nums_list)
        sums.append(sum(nums_list))
sums.reverse()
table_rows_str = "\n".join(table_rows)
contents+=table_rows_str + "\n" + f"###### Updated {datetime.now()}"
Markdown(contents)

row's len = 150


###### Updated 2025-09-26 16:41:01.057916

### Metrics
|round|numbers|bonus|sum|0|10|20|30|40|
|--|--|--|--|--|--|--|--|--|
|1191|07,09,19,23,26,45|33|129|2|1|2|0|1|
|1190|07,09,19,23,26,45|33|129|2|1|2|0|1|
|1189|09,19,29,35,37,38|31|167|1|1|1|3|0|
|1188|03,04,12,19,22,27|9|87|2|2|2|0|0|
|1187|05,13,26,29,37,40|42|150|1|1|2|1|1|
|1186|02,08,13,16,23,28|35|90|2|2|2|0|0|
|1185|06,17,22,28,29,32|38|134|1|1|3|1|0|
|1184|14,16,23,25,31,37|42|146|0|2|2|2|0|
|1183|04,15,17,23,27,36|31|122|1|2|2|1|0|
|1182|01,13,21,25,28,31|22|119|1|1|3|1|0|
|1181|08,10,14,20,33,41|28|126|1|2|1|1|1|
|1180|06,12,18,37,40,41|3|154|1|2|0|1|2|
|1179|03,16,18,24,40,44|21|145|1|2|1|0|2|
|1178|05,06,11,27,43,44|17|136|2|1|1|0|2|
|1177|03,07,15,16,19,43|21|103|2|3|0|0|1|
|1176|07,09,11,21,30,35|29|113|2|1|1|2|0|
|1175|03,04,06,08,32,42|31|95|4|0|0|1|1|
|1174|08,11,14,17,36,39|22|125|1|3|0|2|0|
|1173|01,05,18,20,30,35|3|109|2|1|1|2|0|
|1172|07,09,24,40,42,44|45|166|2|0|1|0|3|
|1171|03,06,07,11,12,17|19|56|3|3|0|0|0|
|1170|03,13,28,34,38,42|25|158|1|1|1|2|1|
|1169|05,12,24,26,39,42|20|148|1|1|2|1|1|
|1168|09,21,24,30,33,37|29|154|1|0|2|3|0|
|1167|08,23,31,35,39,40|24|176|1|0|1|3|1|
|1166|14,23,25,27,29,42|16|160|0|1|4|0|1|
|1165|06,07,27,29,38,45|17|152|2|0|2|1|1|
|1164|17,18,23,25,38,39|22|160|0|2|2|2|0|
|1163|02,13,15,16,33,43|4|122|1|3|0|1|1|
|1162|20,21,22,25,28,29|6|145|0|0|6|0|0|
###### Updated 2025-09-26 16:41:01.066257

In [6]:
"""
Data preparation
"""

#########
# nums_t
nums_by_row = [[] for i in range(6)]
for nums in nums_t:
    for i in range(6):
        nums_by_row[i].append(nums[i])
for i in range(6):
    nums_by_row[i].reverse()

##########
# row_list
row_list = []
row_list_none_last = []
for nums in nums_by_row:
    row_list.append(nums[0:-1])
    row_list_none_last.append(nums[0: len(nums)-1])

In [7]:
predicted_nums = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0}

In [6]:
def get_predicted(version: str):
    query = f'select version, n0, n1, n2, n3, n4, n5, level from candidated where version like \\"{version_prefix[0]}%\\" order by version asc'
    predict_lines = []    
    rows = !echo "{query}" | sqlite3 ../db/metrics.db
    return rows


def row_to_cols(row: str):
    """ row_to_cols """
    cols = row.split('|')
    level = cols[len(cols)-1]
    version_id = int(cols[0].split(version_prefix[0])[1])
    nums = [int(cols[i]) for i in range(1, len(cols) - 1)]
    sum_val = sum(nums)
    return level, version_id, nums, sum_val

def get_predicted_strs(version: str):
    """ get_predicted_strs """
    rows = get_predicted(version = version)
    predicted_val = []
    for row in rows:
        level, version_id, nums, sum_val = row_to_cols(row)
        nums_str = []
        for num in nums:
            nums_str.append(f'{num: >2}')
        predicted_val.append((f'[{", ".join(nums_str)})]', f'{sum_val: >3}', f'{version_id:02}', f'*{level}'))
    sorted_data_by_col1 = sorted(predicted_val, key=lambda x: x[2])
    return sorted_data_by_col1


def get_predicted_list(version: str):
    """ get_predicted """
    predicted = []
    rows = get_predicted(version = version)
    for row in rows:
        predicted.append(row_to_cols(row))
    return predicted


def print_results_by_round(rounds: list):
    """ get_results_by_round """
    sel_query = f'select * from results where round in ({",".join(rounds)}) order by round asc;'
    rows = !echo "{sel_query}" | sqlite3 ../db/metrics.db
    for row in rows:
        cols = row.split('|')
        nums = [int(i) for i in cols[2].split(',')]
        print(f'{cols[1]} | {nums[0]: >2},{nums[1]: >2},{nums[2]: >2},{nums[3]: >2},{nums[4]: >2},{nums[5]: >2} | {sum(nums)}')
    print('\n')

In [8]:
version_prefixes=[("memo_0901_", 1188), ("memo_0908_", 1189), ("memo_0915_", 1190), ("memo_0926_", 1191), ]
predict_dict = {}
for version_prefix in version_prefixes:
    predict_lines = get_predicted_strs(version_prefix)
    predict_dict[version_prefix] = predict_lines

for k, vs in predict_dict.items():
    print(k, '=')
    for v in vs:
        print(' '*8, v)
        
    print('-'*52)

round_vps = []
for vp in version_prefixes:
    round_vps.append(str(vp[1]))

print_results_by_round(round_vps)

('memo_0901_', 1188) =
         ('[11, 13, 17, 19, 26, 42)]', '128', '01', '*21')
         ('[ 6, 12, 15, 26, 37, 45)]', '141', '02', '*1')
         ('[ 3, 12, 16, 32, 34, 39)]', '136', '03', '*2')
         ('[ 3, 10, 14, 18, 36, 43)]', '124', '04', '*1')
         ('[ 8, 14, 18, 25, 35, 40)]', '140', '05', '*2')
         ('[ 2,  3, 12, 20, 28, 33)]', ' 98', '06', '*22')
         ('[ 5, 10, 12, 23, 33, 44)]', '127', '07', '*22')
         ('[ 3, 10, 14, 18, 36, 43)]', '124', '08', '*0')
----------------------------------------------------
('memo_0908_', 1189) =
         ('[ 9, 14, 22, 25, 27, 37)]', '134', '01', '*21')
         ('[ 3,  4, 12, 19, 22, 27)]', ' 87', '02', '*0')
         ('[ 4, 13, 14, 30, 34, 38)]', '133', '03', '*22')
         ('[ 2,  9, 11, 14, 33, 37)]', '106', '04', '*22')
         ('[ 2, 11, 16, 20, 28, 33)]', '110', '05', '*22')
         ('[ 3, 11, 18, 31, 34, 41)]', '138', '06', '*2')
         ('[ 7,  9, 23, 35, 39, 42)]', '155', '07', '*22')
         ('[ 5, 10, 12,

In [11]:
"""
9,19,29,35,37,38 31

37 29 19 9 38 35  31
"""
query = "insert into results_order(round, n1, n2, n3, n4, n5, n6, bonus) values(1189, 37, 29, 19, 9, 38, 35, 31);"
print(f'{query}')
!echo "{query}"
!echo "{query}" > sqlite3 ../db/metrics.db

insert into results_order(round, n1, n2, n3, n4, n5, n6, bonus) values(1189, 37, 29, 19, 9, 38, 35, 31);
insert into results_order(round, n1, n2, n3, n4, n5, n6, bonus) values(1189, 37, 29, 19, 9, 38, 35, 31);


In [77]:
version_prefixes=[("memo_0811_", 1185), ("memo_0818_", 1186), ("memo_0825_", 1187), ("memo_0901_", 1188), ("memo_0908_", 1189)]
for version_prefix in version_prefixes:
    predicts = get_predicted_list(version_prefix[0])
    freq_dicts = {}
    for predict in predicts:
        for num in predict[2]:
            if num in freq_dicts:
                freq_dicts[num] += 1
            else:
                freq_dicts[num] = 1
    sotred_freq = sorted(freq_dicts.items(), key=lambda item: item[1])
    print(version_prefix, sotred_freq)

('memo_0811_', 1185) [(2, 1), (11, 1), (34, 1), (27, 1), (43, 1), (13, 1), (24, 1), (17, 1), (26, 1), (41, 1), (7, 1), (18, 1), (33, 1), (42, 1), (15, 1), (38, 1), (45, 1), (29, 2), (12, 2), (32, 2), (30, 2), (37, 2), (25, 2), (1, 2), (10, 2), (9, 3), (28, 3), (3, 3), (6, 3), (16, 3), (36, 3), (14, 3)]
('memo_0818_', 1186) [(2, 1), (4, 1), (17, 1), (25, 1), (40, 1), (1, 1), (9, 1), (23, 1), (13, 1), (21, 1), (26, 1), (38, 1), (42, 1), (5, 1), (15, 1), (22, 1), (39, 1), (3, 1), (10, 2), (16, 2), (18, 2), (28, 2), (30, 2), (11, 2), (14, 2), (37, 2), (7, 2), (33, 3), (27, 3)]
('memo_0825_', 1187) [(10, 1), (16, 1), (30, 1), (25, 1), (39, 1), (45, 1), (3, 1), (12, 1), (41, 1), (42, 1), (44, 1), (5, 1), (20, 1), (6, 1), (2, 1), (33, 2), (32, 2), (38, 2), (9, 2), (13, 2), (18, 3), (28, 3), (31, 3), (19, 3), (1, 3), (8, 3), (27, 3), (37, 4), (11, 4)]
('memo_0901_', 1188) [(11, 1), (13, 1), (17, 1), (19, 1), (42, 1), (6, 1), (15, 1), (37, 1), (45, 1), (16, 1), (32, 1), (34, 1), (39, 1), (8, 1)

In [55]:
print('current working date : ', datetime.now())

current working date :  2025-09-10 11:25:01.061945


In [56]:
""" 

1190회 당첨번호 7,9,19,23,26,45+33
23 26 7 19 9 45  33

insert into results_order(round, n1, n2, n3, n4, n5, n6, bonus) values(1190, 23, 26, 7, 19, 9, 45, 33); 

table|results_order|results_order|15|CREATE TABLE results_order(id INTEGER PRIMARY KEY AUTOINCREMENT, round short, n1 short, n2 short, n3 short, n4 short, n5 short, n6 short, bonus short)


update results set metric='3,4,12,19,22,27', bonus=9 where round=1188;
insert into results(round, metric, bonus) values( 1188, '3,4,12,19,22,27', 9);
38, 29, 10, 22, 31, 7, 15
n1=3, n2=19, n3=12, n4=22, n5=4, n6=27, bonus=9
"""

" \n3 4 12 19 22 27. 보너스. 9. \n\nupdate results set metric='3,4,12,19,22,27', bonus=9 where round=1188;\ninsert into results(round, metric, bonus) values( 1188, '3,4,12,19,22,27', 9);\n38, 29, 10, 22, 31, 7, 15\nn1=3, n2=19, n3=12, n4=22, n5=4, n6=27, bonus=9\n"

In [ ]:
## original
('[ 3,  4, 12, 19, 22, 27)]', ' 87', '02', '*0')

('[ 1,  9, 28, 30, 32, 36)]', '136', '03', '*2')
('[ 2, 11, 16, 20, 34, 38)]', '121', '05', '*2')
('[ 2,  3, 12, 20, 28, 33)]', ' 98', '10', '*3')

#----
('[ 3,  4, 12, 19, 22, 27)]', ' 87', '02', '*0')
#----

('[ 3,  9, 11, 14, 32, 36)]', '105', '04', '*2')
('[ 3, 11, 18, 31, 34, 41)]', '138', '06', '*2')

('[ 5, 10, 12, 23, 33, 44)]', '127', '08', '*3')

('[ 7, 12, 31, 35, 39, 42)]', '166', '07', '*2')
('[ 8, 14, 18, 25, 35, 40)]', '140', '09', '*3')
('[ 9, 14, 22, 25, 27, 37)]', '134', '01', '*1')
